In [129]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

from keras.datasets import reuters

In [3]:
# Import netflow capture file

flowdata = pd.read_csv("capture20110810.binetflow", sep=',', header=0)

# Strip whitespace
flowdata.rename(columns=lambda x: x.strip(), inplace = True)

In [104]:
flowdata_sample_concat = pd.concat([flowdata_sample.ix[:,2:5], flowdata_sample.ix[:,6:8], flowdata_sample.ix[:,11:14]], axis=1)

flowdata_sample_concat.Proto = flowdata_sample_concat.Proto.astype(str)
flowdata_sample_concat.SrcAddr = flowdata_sample_concat.SrcAddr.astype(str)
flowdata_sample_concat.DstAddr = flowdata_sample_concat.DstAddr.astype(str)
flowdata_sample_concat.Sport = flowdata_sample_concat.Sport.astype(str)
flowdata_sample_concat.Dport = flowdata_sample_concat.Dport.astype(str)
flowdata_sample_concat.TotPkts = flowdata_sample_concat.TotPkts.astype(int)
flowdata_sample_concat.TotBytes = flowdata_sample_concat.TotBytes.astype(int)
flowdata_sample_concat.SrcBytes = flowdata_sample_concat.SrcBytes.astype(int)



In [105]:
flowdata_sample_concat

,Proto,SrcAddr,Sport,DstAddr,Dport,TotPkts,TotBytes,SrcBytes
929053,udp,124.190.252.82,41911,147.32.84.229,13363,4,274,154
358975,udp,147.32.84.138,35119,147.32.80.9,53,2,216,83
1101845,udp,147.32.84.138,53532,147.32.80.9,53,2,214,81
1971975,udp,188.27.145.146,53889,147.32.84.229,13363,4,817,143
1530286,udp,99.99.136.224,24589,147.32.84.229,13363,4,783,154
317020,udp,147.32.84.138,58755,147.32.80.9,53,2,214,81
1448648,udp,147.32.84.138,38523,147.32.80.9,53,2,214,81
1996394,udp,147.32.84.229,13363,178.141.109.155,54351,6,843,221
648065,udp,147.32.84.138,49478,147.32.80.9,53,2,214,81
1168171,udp,212.166.149.100,3358,147.32.84.229,13363,2,162,69


In [111]:
# Create random sample for testing

flowdata_sample = flowdata.sample(n = 500)

# Convert TimeStamp to POSIX format

flowdata_sample['StartTime'] = pd.to_datetime(flowdata_sample['StartTime'], unit='ms')

# Drop NAs in the dataset

flowdata_sample = flowdata_sample.dropna(axis=0, how='any')


# Create testing and training sets

flowdata_dict = pd.DataFrame.to_dict(flowdata_sample_concat)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [112]:
vec = DictVectorizer()

trans_vec = vec.fit_transform(flowdata_dict.values()).toarray()

In [127]:
maxlen = 100
max_features = 2000

# Build model 
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128))  # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop')

model.fit(X_train, Y_train, batch_size=16, nb_epoch=10)
score = model.evaluate(X_test, Y_test, batch_size=16)

Build model...


TypeError: __init__() takes at least 3 arguments (2 given)

In [130]:
max_words = 1000
batch_size = 32
nb_epoch = 5

print("Loading data...")
(X_train, y_train), (X_test, y_test) = reuters.load_data(nb_words=max_words, test_split=0.2)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
9216000/9211982 [==============================] - 2s     
(8982, 'train sequences')
(2246, 'test sequences')


In [134]:
word_index = reuters.get_word_index(path='reuters_word_index.pkl')

In [139]:
word_index["a"]

7